In [70]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [71]:
# allow for appending different runs' results.

#folder_path = "/Users/oas/Downloads"
folder_path = "/Users/oas/Documents/VOD/vod/benchmarking_results"
files = [
    os.path.join(folder_path, file)
    for file in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, file))
]
newest_file = max(files, key=os.path.getmtime)

print(newest_file)

/Users/oas/Documents/VOD/vod/benchmarking_results/2024-02-06_06:09:43.csv


In [72]:
df_results = pd.read_csv(newest_file)

In [73]:
df_results

,Unnamed: 0,Dataset,Index,Index parameters.,Server startup speed (s),Index build speed (s),Search speed avg. (ms),Search speed p95 (ms),Recall avg,Recall@1,Recall@10,Recall@100,ingesting_max,ingesting_mean,benchmarking_max,benchmarking_mean
0,0,sift-128-euclidean,"index: faiss IVF, n_partition=500, None, L2","IVF, n_partition=500, None, L2",7.759579,6.398616,13.329702,16.054447,0.936767,0.932333,0.991000,0.991333,529.71,529.552857,529.71,528.954444
1,1,sift-128-euclidean,"index: faiss IVF, n_partition=2000, None, L2","IVF, n_partition=2000, None, L2",-16.044085,54.003787,13.360078,15.766835,0.947160,0.939000,0.998667,0.999000,530.33,529.899333,531.31,530.899615
2,2,sift-128-euclidean,None,error: Traceback (most recent call last):\n F...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.000000,-1.00,-1.000000
3,3,sift-128-euclidean,None,error: Traceback (most recent call last):\n F...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.000000,-1.00,-1.000000
4,4,sift-128-euclidean,None,error: Traceback (most recent call last):\n F...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.000000,-1.00,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,157,lastfm-64-dot,None,error: Traceback (most recent call last):\n F...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.000000,-1.00,-1.000000
158,158,lastfm-64-dot,None,error: Traceback (most recent call last):\n F...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00,-1.000000,-1.00,-1.000000
159,159,lastfm-64-dot,"index: qdrant HNSW, M=64, ef_construction=256,...","HNSW, M=64, ef_construction=256, ef_search=128...",93.669671,11.948119,108.071994,356.787806,1.000000,1.000000,1.000000,1.000000,560.25,469.273333,561.65,519.984615
160,160,lastfm-64-dot,"index: qdrant HNSW, M=128, ef_construction=256...","HNSW, M=128, ef_construction=256, ef_search=25...",92.125881,11.902588,107.035071,355.746668,1.000000,1.000000,1.000000,1.000000,577.15,486.875000,562.05,513.833077


In [74]:
df_results = df_results.query("Index != 'None'")  # ignore failed runs
df_results = df_results.query("benchmarking_mean != -1")  # ignore failed runs

In [75]:
df_results["index_prov"] = df_results.Index.str.split(" ").apply(lambda l: l[1])

In [76]:
df_results.columns

Index(['Unnamed: 0', 'Dataset', 'Index', 'Index parameters.',
       'Server startup speed (s)', 'Index build speed (s)',
       'Search speed avg. (ms)', 'Search speed p95 (ms)', 'Recall avg',
       'Recall@1', 'Recall@10', 'Recall@100', 'ingesting_max',
       'ingesting_mean', 'benchmarking_max', 'benchmarking_mean',
       'index_prov'],
      dtype='object')

In [88]:
# df_results[df_results.isna()]
df_results.

Unnamed: 0                  False
Dataset                     False
Index                       False
Index parameters.           False
Server startup speed (s)    False
Index build speed (s)       False
Search speed avg. (ms)      False
Search speed p95 (ms)       False
Recall avg                  False
Recall@1                    False
Recall@10                   False
Recall@100                  False
ingesting_max               False
ingesting_mean              False
benchmarking_max             True
benchmarking_mean            True
index_prov                  False
dtype: bool

In [79]:
import plotly.express as px
import plotly.graph_objects as go

# Your existing DataFrame and plot
fig = px.scatter(
    df_results,
    x="Recall avg",
    y="Search speed avg. (ms)",
    hover_data=["Index","ingesting_max","benchmarking_mean"],
    color="index_prov",
    size=df_results["benchmarking_mean"]**2,
)

# # Adding a similar scatter plot but with semi-transparency and different size mapping
# fig.add_trace(
#     go.Scatter(
#         x=df_results["Recall avg"],
#         y=df_results["Search speed avg. (ms)"],
#         mode="markers",
#         marker=dict(
#             size=df_results["ingesting_max"]/25,  # Size based on 'ingesting_max' column
#             opacity=0.1,  # Semi-transparent markers
#             #color=fig.data[0].marker.color,  # Use the same color as the first plot
#             #line=dict(color="MediumPurple", width=2),
#         ),
#         hoverinfo="text",  # You can customize hover info as needed
#         name="Ingesting max",
#     )
# )

fig.show()

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [nan, nan]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above